In [45]:
import numpy as np

from keras.layers import *
from keras.models import Sequential
from keras.utils import np_utils

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

def create_model(x_train, y_train, x_test, y_test):
    from keras import backend as K
    def f1(y_true, y_pred):
        def recall(y_true, y_pred):
            """Recall metric.

            Only computes a batch-wise average of recall.

            Computes the recall, a metric for multi-label classification of
            how many relevant items are selected.
            """
            true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
            possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
            recall = true_positives / (possible_positives + K.epsilon())
            return recall

        def precision(y_true, y_pred):
            """Precision metric.

            Only computes a batch-wise average of precision.

            Computes the precision, a metric for multi-label classification of
            how many selected items are relevant.
            """
            true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
            predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
            precision = true_positives / (predicted_positives + K.epsilon())
            return precision
        precision = precision(y_true, y_pred)
        recall = recall(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))



    model = Sequential()
    model.add(Conv2D({{choice([32,64,128,256, 512,1024])}}, (3, 3), input_shape=x_train.shape[1:], strides=(1, 1), padding="valid"))
    #model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Conv2D({{choice([32,64,128,256, 512,1024])}}, (3, 3), strides=(1, 1), padding="valid"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout({{uniform(0, 1)}}))

    if {{choice(['three', 'four'])}} == 'four':
        model.add(Conv2D({{choice([32,64,128,256, 512,1024])}}, (3, 3), strides=(1, 1), padding="valid"))
        model.add(Activation('relu'))
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout({{uniform(0, 1)}}))

    #model.add(Conv2D(256, (3, 3), strides=(1, 1), activation="relu", padding="valid"))
    model.add(Flatten())
    #model.add(GlobalAveragePooling2D())

    model.add(Dense({{choice([32,64,128,256, 512,1024])}}))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc',f1])

    result = model.fit(x_train, y_train,
                  batch_size={{choice([64, 128,256])}},
                  epochs=60,
                  verbose=0,
                  validation_split=0.1)


    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_f1']) 
    print('Best validation metric of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score,f1_score, confusion_matrix, mean_absolute_error,mean_squared_error, median_absolute_error
from sklearn.preprocessing import normalize
import time
import matplotlib.pyplot as plt

def plot_df(df):
    try:
        from IPython.display import display
        display(df)
    except:
        print(df)
    
def plot_confusion_matrix(cm, target_names, title='Confusion matrix (f1-score)',cmap=None, normalize=True):
    import itertools
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0.0, vmax=1.0)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    
def calculate_metrics(y_true,y_pred,plot=True, title=""):
    dic_return = {}
    dic_return["Precision raw"] = precision_score(y_true,y_pred,average=None,labels=[0,1])
    dic_return["Recall raw"] = recall_score(y_true,y_pred,average=None,labels=[0,1])
    #dic_return["Precision"] = precision_score(y_true,y_pred,average=None,labels=[0,1])
    #dic_return["Recall"] = recall_score(y_true,y_pred,average=None,labels=[0,1])
    dic_return["F1 raw"] = f1_score(y_true,y_pred,average=None,labels=[0,1])
    dic_return["F1 weighted"] = f1_score(y_true,y_pred,average="weighted",labels=[0,1])
    dic_return["F1 macro"] = f1_score(y_true,y_pred,average="macro",labels=[0,1])
    dic_return["F1 micro"] = f1_score(y_true,y_pred,average="micro",labels=[0,1])
    matriz=confusion_matrix(y_true,y_pred, labels=[0,1])
    if plot:
        df = pd.DataFrame(dic_return)
        df.index = ["False Positive","Confirmed"]
        plot_df(df)
        plot_confusion_matrix(normalize(matriz,axis=1,norm='l1'),["False Positive","Confirmed"],title)
    dic_return["Confusion Matrix"] = matriz
    return dic_return


def calculate_median_abs_err(real, pred): 
    if len(real.shape) > 1:
        return np.mean([median_absolute_error(real[:,d],pred[:,d]) for d in range(real.shape[1])])   
    else:
        return median_absolute_error(real,pred)
    
def calculate_mean_abs_perce_err(real, pred):
    diff = np.abs((real - pred) / np.clip(np.abs(real), 1e-7, None))
    return 100. * np.mean(diff) #sin *100 es "fractional"

def calculate_Rmean_squar_log_err(real, pred):
    first_log = np.log(np.clip(pred, 1e-7, None) + 1.)
    second_log = np.log(np.clip(real, 1e-7, None) + 1.)
    return np.sqrt(np.mean(np.square(first_log - second_log)))

def evaluate_metadata(real, pred, plot=False):
    dic_res = {}
    dic_res["MSE"] = mean_squared_error(real, pred)
    dic_res["MAE"] = mean_absolute_error(real, pred)
    dic_res["MeAE"] = calculate_median_abs_err(real, pred)
    dic_res["MApE"] = calculate_mean_abs_perce_err(real,pred)
    dic_res["RMSLE"] = calculate_Rmean_squar_log_err(real,pred)
    if plot:
        df = pd.DataFrame(dic_res)
        df.index = ["Real"]
        plot_df(df)
    return dic_res

def evaluate_metadata_raw(real, pred, plot=True, metadata_used=[""]):
    mse_raw = []
    mae_raw = []
    mEae_raw = []
    maPe_raw = []
    rmsle_raw = []
    for d in range(real.shape[1]):        
        dic_aux = evaluate_metadata(real[:,d], pred[:,d], plot=False)
        mse_raw.append(dic_aux["MSE"])
        mae_raw.append(dic_aux["MAE"])
        mEae_raw.append(dic_aux["MeAE"])
        maPe_raw.append(dic_aux["MApE"])
        rmsle_raw.append(dic_aux["RMSLE"])
        
    res_metadata = {}
    res_metadata["MSE"] = mse_raw
    res_metadata["MAE"] = mae_raw
    res_metadata["MeAE"] = mEae_raw
    res_metadata["MApE"] = maPe_raw
    res_metadata["RMSLE"] = rmsle_raw
    
    if plot:
        df = pd.DataFrame(res_metadata)
        df.index = metadata_used
        df.loc["Global"] = np.mean(df, axis=0)
        plot_df(df.transpose())
    res_metadata["MSE"].append(np.mean(mse_raw))
    res_metadata["MAE"].append(np.mean(mae_raw))
    res_metadata["MeAE"].append(np.mean(mEae_raw))
    res_metadata["MApE"].append(np.mean(maPe_raw))
    res_metadata["RMSLE"].append(np.mean(rmsle_raw))
    return res_metadata




def train_model(aux_model, x_train, y_train, x_val, y_val, e=25, BS=128):
    hist=aux_model.fit(x_train, y_train, batch_size=BS, epochs=e, verbose=1, validation_data=(x_val,y_val))
    plt.plot(hist.history['loss'],label="train_loss")
    plt.plot(hist.history['val_loss'],label="val_loss")
    plt.title("Model Loss BCE CNN") 
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend(loc="upper right")
    plt.show()

    plt.plot(hist.history['acc'],label="Train accuracy" )
    plt.plot(hist.history['val_acc'],label="Val accuracy" )
    plt.title("Model Accuracy BCE CNN") 
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend(loc="upper right")
    plt.show()

    predichos=aux_model.predict_classes(x_val)
    calculate_metrics(y_val, predichos,'[VAL] Confusion Matrix 50 epochs CNN + reverse LC')


In [4]:
df_aux = pd.read_csv("../../KOI_Data/kepler_dataset.csv")
labels = (df_aux["NExScI Disposition"] == "CONFIRMED").values*1
labels.shape

(8054,)

In [5]:
channel_1 = np.load('/work/work_teamEXOPLANET/MTF_margarita/no_invertidos/MTF_u15_d15.npy')  
channel_2 = np.load('/work/work_teamEXOPLANET/MTF_margarita/invertidos/MTF_u15_d15.npy')  
channel_3 = np.load('/work/work_teamEXOPLANET/MTF_gabo/npys/time_channel_30.npy')  

In [6]:
x_all = []
for i in range(channel_1.shape[0]):
    combined_image = np.dstack((channel_1[i],channel_2[i],channel_3[i]))
    x_all.append(combined_image)
x_all = np.asarray(x_all)
x_all.shape

(8054, 30, 30, 3)

In [7]:
fold = "../../KOI_Data/"
df_sets = pd.read_csv(fold+"/koi_sets.csv") 
mask_train = (df_sets["Set"] == "Train").values
mask_test = (df_sets["Set"] == "Test").values
mask_unlabeled = (df_sets["Set"] == "Unlabeled").values
df_sets.head()

,KOI Name,Set
0,K00889.01,Train
1,K01009.01,Train
2,K07621.01,Test
3,K06252.01,Test
4,K00958.01,Unlabeled


In [8]:
df_meta = pd.read_csv(fold+"/koi_metadata.csv")
df_meta_train = df_meta[mask_train]
df_meta_test = df_meta[mask_test]
df_meta_unb = df_meta[mask_unlabeled]

df_meta_train.head()

,KOI Name,NExScI Disposition,Kepler Name,Period,Time of Transit Epoch,First Transit Time (BJD),Impact Parameter,Duration,Transit Depth,r/R,...,Teff,log(g),Metallicity,Stellar Radius,Stellar Mass,Max single event sigma,Max Multievent sigma,Transit Model SNR,KOI count,Transit Number
0,K00889.01,CONFIRMED,Kepler-75 b,8.884923,169.991686,2455002.992,0.069,2.07004,16053.4,0.114,...,5332.0,4.500,-0.08,0.843,0.821,61.291,474.021,388.2,1,153
1,K01009.01,FALSE POSITIVE,NaN,5.092465,357.535940,2455190.536,0.161,3.65900,254.0,0.014,...,5037.0,4.547,-0.20,0.751,0.727,3.648,15.175,17.1,1,225
7,K07622.01,FALSE POSITIVE,NaN,36.267167,155.081300,2454988.081,0.214,2.06500,627.1,0.022,...,4264.0,4.637,-0.20,0.620,0.607,3.016,7.472,10.0,1,39
9,K00998.01,FALSE POSITIVE,NaN,161.788327,214.039000,2455047.039,0.267,5.31000,87750.0,0.273,...,6018.0,4.327,0.18,1.212,1.142,341.666,806.993,704.5,1,9
10,K00992.01,CONFIRMED,Kepler-745 b,9.931463,136.491280,2454969.491,0.037,5.67800,398.6,0.018,...,5782.0,4.280,0.16,1.220,1.035,4.530,22.901,26.2,2,137


In [9]:
y_train= (df_meta_train['NExScI Disposition'].values=="CONFIRMED")*1
y_val = (df_meta_test['NExScI Disposition'].values=="CONFIRMED")*1

unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_val, counts_val = np.unique(y_val, return_counts=True)

print('y_train distribution: 0:%i   1:%i'%(counts_train[0],counts_train[1]))
print('y_val distribution:   0:%i   1:%i'%(counts_val[0],counts_val[1]))

y_train distribution: 0:2729   1:1610
y_val distribution:   0:1247   1:671


In [10]:
x_train = x_all[mask_train]
x_val = x_all[mask_test]

if x_train.shape[0] < y_train.shape[0]:
    y_train = y_train[:x_train.shape[0]]
    
print ("Objetos de entrenamiento:",x_train.shape, y_train.shape)
print ("Objetos de validación:",x_val.shape,y_val.shape)

Objetos de entrenamiento: (4339, 30, 30, 3) (4339,)
Objetos de validación: (1918, 30, 30, 3) (1918,)


In [21]:
def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    channel_1 = np.load('/work/work_teamEXOPLANET/MTF_margarita/no_invertidos/MTF_u15_d15.npy')  
    channel_2 = np.load('/work/work_teamEXOPLANET/MTF_margarita/invertidos/MTF_u15_d15.npy')  
    channel_3 = np.load('/work/work_teamEXOPLANET/MTF_gabo/npys/time_channel_30.npy')  

    x_all = []
    for i in range(channel_1.shape[0]):
        combined_image = np.dstack((channel_1[i],channel_2[i],channel_3[i]))
        x_all.append(combined_image)
    x_all = np.asarray(x_all)

    fold = "../../KOI_Data/"
    df_sets = pd.read_csv(fold+"/koi_sets.csv") 
    mask_train = (df_sets["Set"] == "Train").values
    mask_test = (df_sets["Set"] == "Test").values

    df_meta = pd.read_csv(fold+"/koi_metadata.csv")
    df_meta_train = df_meta[mask_train]
    df_meta_test = df_meta[mask_test]
    y_train= (df_meta_train['NExScI Disposition'].values=="CONFIRMED")*1
    y_val = (df_meta_test['NExScI Disposition'].values=="CONFIRMED")*1

    unique_train, counts_train = np.unique(y_train, return_counts=True)
    unique_val, counts_val = np.unique(y_val, return_counts=True)

    x_train = x_all[mask_train]
    x_val = x_all[mask_test]
    
    return x_train, y_train, x_val, y_val

In [47]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=40,
                                          trials=Trials(),
                                     notebook_name='Hyperkeras Best Model')
print("Evalutation of best performing model:")
print(best_model.evaluate(x_val, y_val))
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    from keras.layers import *
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.model_selection import KFold
except:
    pass

try:
    from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, mean_absolute_error, mean_squared_error, median_absolute_error
except:
    pass

try:
    from sklearn.preprocessing import normalize
except:
    pass

try:
    import time
except:
    pass

try:
    import

_________________________________________________________________
conv2d_23 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
activation_30 (Activation)   (None, 4, 4, 64)          0         
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1024)              263168    
_________________________________________________________________
activation_31 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_31

Total params: 2,549,313                                                          
Trainable params: 2,549,313                                                      
Non-trainable params: 0                                                          
_________________________________________________________________                
Best validation metric of epoch:                                                 
0.6557555748020998                                                               
Model: "sequential_13"                                                           
_________________________________________________________________                
Layer (type)                 Output Shape              Param #                   
conv2d_30 (Conv2D)           (None, 28, 28, 64)        1792                      
_________________________________________________________________                
activation_40 (Activation)   (None, 28, 28, 64)        0                         
________________

activation_48 (Activation)   (None, 12, 12, 64)        0                        
_________________________________________________________________               
max_pooling2d_36 (MaxPooling (None, 6, 6, 64)          0                        
_________________________________________________________________               
dropout_48 (Dropout)         (None, 6, 6, 64)          0                        
_________________________________________________________________               
conv2d_37 (Conv2D)           (None, 4, 4, 128)         73856                    
_________________________________________________________________               
activation_49 (Activation)   (None, 4, 4, 128)         0                        
_________________________________________________________________               
max_pooling2d_37 (MaxPooling (None, 2, 2, 128)         0                        
_________________________________________________________________               
dropout_49 (Dropout)        

_________________________________________________________________               
dense_30 (Dense)             (None, 1)                 33                       
Total params: 5,337,665                                                         
Trainable params: 5,337,665                                                     
Non-trainable params: 0                                                         
_________________________________________________________________               
Best validation metric of epoch:                                                
0.1337696837802087                                                              
Model: "sequential_18"                                                          
_________________________________________________________________               
Layer (type)                 Output Shape              Param #                  
conv2d_43 (Conv2D)           (None, 28, 28, 64)        1792                     
____________________________

...

_________________________________________________________________                  
max_pooling2d_90 (MaxPooling (None, 2, 2, 1024)        0                           
_________________________________________________________________                  
dropout_124 (Dropout)        (None, 2, 2, 1024)        0                           
_________________________________________________________________                  
flatten_35 (Flatten)         (None, 4096)              0                           
_________________________________________________________________                  
dense_69 (Dense)             (None, 1024)              4195328                     
_________________________________________________________________                  
activation_125 (Activation)  (None, 1024)              0                           
_________________________________________________________________                  
dropout_125 (Dropout)        (None, 1024)              0                    

Non-trainable params: 0                                                            
_________________________________________________________________                  
Best validation metric of epoch:                                                   
0.7565377840798022                                                                 
Model: "sequential_40"                                                             
_________________________________________________________________                  
Layer (type)                 Output Shape              Param #                     
conv2d_96 (Conv2D)           (None, 28, 28, 512)       14336                       
_________________________________________________________________                  
activation_133 (Activation)  (None, 28, 28, 512)       0                           
_________________________________________________________________                  
max_pooling2d_96 (MaxPooling (None, 14, 14, 512)       0                    

activation_141 (Activation)  (None, 28, 28, 256)       0                         
_________________________________________________________________                
max_pooling2d_102 (MaxPoolin (None, 14, 14, 256)       0                         
_________________________________________________________________                
dropout_141 (Dropout)        (None, 14, 14, 256)       0                         
_________________________________________________________________                
conv2d_103 (Conv2D)          (None, 12, 12, 32)        73760                     
_________________________________________________________________                
activation_142 (Activation)  (None, 12, 12, 32)        0                         
_________________________________________________________________                
max_pooling2d_103 (MaxPoolin (None, 6, 6, 32)          0                         
_________________________________________________________________                
dropout_142 (Dro

_________________________________________________________________                
dropout_150 (Dropout)        (None, 2, 2, 1024)        0                         
_________________________________________________________________                
flatten_42 (Flatten)         (None, 4096)              0                         
_________________________________________________________________                
dense_83 (Dense)             (None, 256)               1048832                   
_________________________________________________________________                
activation_151 (Activation)  (None, 256)               0                         
_________________________________________________________________                
dropout_151 (Dropout)        (None, 256)               0                         
_________________________________________________________________                
dense_84 (Dense)             (None, 1)                 257                       
Total params: 2,

Model: "sequential_48"                                                           
_________________________________________________________________                
Layer (type)                 Output Shape              Param #                   
conv2d_118 (Conv2D)          (None, 28, 28, 32)        896                       
_________________________________________________________________                
activation_163 (Activation)  (None, 28, 28, 32)        0                         
_________________________________________________________________                
max_pooling2d_118 (MaxPoolin (None, 14, 14, 32)        0                         
_________________________________________________________________                
dropout_163 (Dropout)        (None, 14, 14, 32)        0                         
_________________________________________________________________                
conv2d_119 (Conv2D)          (None, 12, 12, 32)        9248                      
________________

In [48]:
best_run

{'Activation': 0,
 'Conv2D': 4,
 'Conv2D_1': 2,
 'Conv2D_2': 3,
 'Conv2D_3': 5,
 'Dropout': 0.6844927856873184,
 'Dropout_1': 0.25438903160284304,
 'Dropout_2': 0,
 'Dropout_3': 0.22527254444045003,
 'Dropout_4': 0.5597328676576347,
 'batch_size': 2}

In [49]:
best_model.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_74 (Conv2D)           (None, 28, 28, 512)       14336     
_________________________________________________________________
activation_102 (Activation)  (None, 28, 28, 512)       0         
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 14, 14, 512)       0         
_________________________________________________________________
dropout_102 (Dropout)        (None, 14, 14, 512)       0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 12, 12, 128)       589952    
_________________________________________________________________
activation_103 (Activation)  (None, 12, 12, 128)       0         
_________________________________________________________________
max_pooling2d_75 (MaxPooling (None, 6, 6, 128)       